In [ ]:
#================================================
#DEBUG TOGGLE SWITCH

def d(*args, **kwargs):
    if d.ON:
        print(*args, **kwargs)

d.ON = True          # <- module-local switch

##example usage
#d("Initial x shape:", x.shape)
#================================================
if __name__ == "__main__":
  try:
    import segmentation_models_pytorch as smp
  except ImportError:
    !pip install -q segmentation_models_pytorch timm
    import segmentation_models_pytorch as smp
#============================================================0


#================================================
# NAVIGATION TO GET TO THE FOLDER WITH STUFF INSIDE
import sys
sys.path.append("/content/drive/MyDrive/MAGISTRALE/ANNO 1/Computer Vision/Project/RoadObstacleDetection/ProjectWorkspace/src")
#=================================================0




#=================================================
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader

from network.deeplab_dualhead import get_model
from dataloader.cityscapes_dataloader import CityscapesDataset
#========================================================================



#========================================================================
# PATHS SETTING
calib_images_dir = "/content/drive/MyDrive/MAGISTRALE/ANNO 1/Computer Vision/Project/RoadObstacleDetection/Datasets/leftImg8bit_trainvaltest/leftImg8bit_trainvaltest/leftImg8bit/val"
calib_masks_dir = "/content/drive/MyDrive/MAGISTRALE/ANNO 1/Computer Vision/Project/RoadObstacleDetection/Datasets/leftImg8bit_trainvaltest/gtFine_trainvaltest/gtFine/val"
split_dir = "/content/drive/MyDrive/MAGISTRALE/ANNO 1/Computer Vision/Project/RoadObstacleDetection/ProjectWorkspace/splits"

with open(os.path.join(split_dir, "calib_cp.txt"), "r") as f:
  calib_list = [line.strip() for line in f]
#===========================================================================


#============================================
# TAKIN DATASET
calib_dataset = CityscapesDataset(calib_images_dir, calib_masks_dir, calib_list, augment=False) #ofc augm was just for training
calib_loader = DataLoader(calib_dataset, batch_size=1, shuffle=False, num_workers=2) #now i want data to be provided one at a time
#==============================================


#===============================================================
# SETTING UP THE MODEL
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_model().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/MAGISTRALE/ANNO 1/Computer Vision/Project/RoadObstacleDetection/ProjectWorkspace/ckpts/updated_loss_model.pth"))
model.eval()
#===================================================================================0


#====================================================================================
# NON CONFORMITY SCORES COMPUTATION
scores = []
with torch.no_grad(): #deactivation of gradient tracking as well for non training purposes
  for images, _, _ in tqdm(calib_loader):
    images = images.to(device)
    pred_softmax, _ = model(images)
    pred_softmax = F.interpolate(pred_softmax, size=images.shape[-2:], mode="bilinear", align_corners=False) #outpu has reduced dimension remember
    max_probs = torch.max(torch.softmax(pred_softmax, dim=1), dim=1)[0] #takin max prob value for each px
    nonconf = 1.0 - max_probs #non conformity score
    scores.append(nonconf.cpu().numpy())

scores = np.concatenate([s.flatten() for s in scores], axis=0) #take all the arrays in scores variable, flatten in 1D and merging in one array with all of em
#====================================================================================



#=========================================================================================
# CONFORMAL CALIBRATION
alpha = 0.15
lambda_hat = np.quantile(scores, 1-alpha) #1-alpha to ensure the 97% of those values are under this treshold -> indicates how much uncertainty has THAT px if it goes over
print(f"Calibrated lambda_hat at alpha = {alpha}: {lambda_hat:.6f}")

with open("/content/drive/MyDrive/MAGISTRALE/ANNO 1/Computer Vision/Project/RoadObstacleDetection/ProjectWorkspace/src/eval/lambda_hat_3rdattempt.txt", "w") as f:
  f.write(f"{lambda_hat:.6f}")
  f.write("\n")
#=========================================================================================



##########################################################################################################
# DUMMY TEST JUST WITH 5 PIXELS FROM FIRST IMAGE

if __name__ == "__main__":
  sample_image, _, _ = calib_dataset[0]
  sample_image = sample_image.unsqueeze(0).to(device)

  with torch.no_grad():
    pred_softmax, _ = model(sample_image)
    pred_softmax = F.interpolate(pred_softmax, size=sample_image.shape[-2:], mode="bilinear", align_corners=False)
    max_probs = torch.max(torch.softmax(pred_softmax, dim=1), dim=1)[0]
    nonconf = 1.0 - max_probs

  d("Image shape:", sample_image.shape)
  flat_scores = nonconf.view(-1)
  d("First 5 non conformity scores:", flat_scores[:5].cpu().numpy())

  with open("/content/drive/MyDrive/MAGISTRALE/ANNO 1/Computer Vision/Project/RoadObstacleDetection/ProjectWorkspace/src/eval/lambda_hat_3rdattempt.txt", "r") as f:
    lambda_hat_loaded = float(f.readline().strip())

  d("Loaded lambda_hat:", lambda_hat_loaded)

  exceed = flat_scores[:5] > lambda_hat_loaded
  d("Let's see if these 5 pixels exceed lambda_hat", exceed.cpu().numpy())












   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.7 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

100%|██████████| 100/100 [01:27<00:00,  1.15it/s]


Calibrated lambda_hat at alpha = 0.15: 0.418286
Image shape: torch.Size([1, 3, 512, 1024])
First 5 non conformity scores: [0.1016537  0.1016537  0.10696512 0.11863667 0.1319462 ]
Loaded lambda_hat: 0.418286
Let's see if these 5 pixels exceed lambda_hat [False False False False False]
